## FFM

[Field-aware Factorization Machines for CTR Prediction](https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf) 论文。

TL;DR;

- 提出了 Field 的概念。
- 训练时梯度计算使用 AdaGrad 方法
- 参考[HOGWILD!: A Lock-Free Approach to Parallelizing Stochastic Gradient Descent](https://arxiv.org/abs/1106.5730) 给了一个并行设计的方案。

关键词：CTR, 计算广告，FM.

### 1. Introduciton

假设有 $m$ 个样本，每个样本有 $n$ 个特征，即:$\{(x_i, y_i)\}_{i=1}^{m}, x_i \in \mathbb R^n$, 则我们的模型为：

$$
\underset{w}{min} \frac{\lambda}{2}\|\mathbf{w}\|_2^2 + \sum_{i=1}^{m} \log \big(1 + exp(-y_i \phi_{LM}(\mathbf{w}, \mathbf{x_i}))\big) \tag{1}
$$

在 $(1)$ 中， $\lambda$ 是正则参数，在损失函数中我们考虑的是一个线性模型：

$$
\phi_{LM}(\mathbf{w}, \mathbf{x}) = \mathbf{w} \cdot \mathbf{x}
$$

| clicked |	 unclicked |	 Publisher |	Advertiser |
|----|-------|--------|------|
|+80 |	 −20 |	 ESPN |	Nike |
|+10 |	 −90 |	 ESPN |	Gucci |
|+0 |	 −1 |	 	 ESPN |	Adidas |
|+15 |	 −85 |	 Vogue |	Nike |
|+90 |	 −10 |	 Vogue |	Gucci |
|+10 |	 −90 |	 Vogue |	Adidas |
|+85 |	 −15 |	 NBC |	Nike |
|+0 |	 −0 |	 	 NBC |	Gucci |
|+90 |	 −10 |	 NBC |	Adidas |

以上面表格的数据为例子，我们可以看到 Cucci 和 Vogue 组合有很高的点击率。但是普通的线性模型中很难能学到相应的特征关系。所以我们需要使用组合策略来对特征进行组合。

常用的组合策略有， 多项式组合和FM。SVM 通过核也可以进行特征组合，但计算性能不高。

### 2. POLY2 和 FM

2 阶多项式的特征组合模型为：

$$
\phi_{Poly2}(\mathbf{w}, \mathbf{x}) = \sum_{j_1 = 1}^n \sum_{j_2 = j_1 + 1}^n w_{h(j_1, j_2)} x_{j_1} x_{j_2}
$$

FM 模型通过学习隐向量来完成特征组合：

$$
\phi_{FM}(\mathbf{w}, \mathbf{x}) = \sum_{j_1 = 1}^n \sum_{j_2 = j_1 + 1}^n (w_{j_1} w_{j_2}) x_{j_1} x_{j_2}
$$

### 3. FFM

FFM 参考了 PITF, 将 Field 的概念引入了进来。以下面的例子来说明：

| clicked |	 Publisher(P) |	 Advertiser(A) |	Gender(G) |
|----|-------|--------|------|
|Yes |	 ESPN |	 Nike |	Make |



如果以 FM 为模型，则 $\phi_{FM}(\mathbf{w}, \mathbf{x})$ 模型如下：

$$
\mathbf{w}_{ESPN} \cdot \mathbf{w}_{Nike} + \mathbf{w}_{ESPN} \cdot \mathbf{w}_{Male} + \mathbf{w}_{Nike} \cdot \mathbf{w}_{Male} 
$$

即每一个特征仅有一个隐向量与其它特征相关联。在 FFMs 中，每一个特征多个隐向量，这取决于特征所属于的 Field. 

在上面的例子中，我们的 $\phi_{FMM}(\mathbf{w}, \mathbf{x})$ 模型如下：

$$
\mathbf{w}_{ESPN, A} \cdot \mathbf{w}_{Nike, P} + \mathbf{w}_{ESPN, G} \cdot \mathbf{w}_{Male, P} + \mathbf{w}_{Nike, G} \cdot \mathbf{w}_{Male, A} 
$$

用数学公式来描述，则用：

$$
\phi_{FMM}(\mathbf{w}, \mathbf{x}) = \sum_{j_1 = 1}^n \sum_{j_2 = j_1 + 1}^n (w_{j_1, f_2} w_{j_2, f_1}) x_{j_1} x_{j_2} \tag{2}
$$

**注意**，特征所属的 Field $f_1, f_2$ 需要交叉学习。

#### 3.1 模型求解

上面的模型没有考虑偏置 $w_0$, 及一阶参数 $w_1, w_2, ..., w_m$。仅考虑组件特征参数的求解。对 $(2)$ 式进行求导，可以得到对应的梯度更新公式：

$$
\mathbf{g}_{j_1, f_2} = \nabla_{w_{j_1}, f_2} f(w) = \lambda \cdot w_{{j_1}, f_2} + \kappa \cdot w_{{j_2}, f_1} x_{j_1} x_{j_2} \tag{5} 
$$

$$
\mathbf{g}_{j_2, f_1} = \nabla_{w_{j_2}, f_1} f(w) = \lambda \cdot w_{{j_2}, f_1} + \kappa \cdot w_{{j_1}, f_2} x_{j_1} x_{j_2} \tag{6} 
$$

其中：

$$
\kappa = \frac{\partial \log(1 + exp(-y \phi_{FFM}(w, x))}{\partial \phi_{FFM}(w, x)} = - \frac{y}{1 + exp(-y \phi_{FFM}(w, x))}
$$

使用 AdaGrad 算法来提升性能，即自适应的来调节学习因子。则有梯度累加平方和为，当 $d =  1, ..., k$ 时，

$$
(G_{j_1, f_2})_d := (G_{j_1, f_2})_d + (g_{j_1, f_2})_d^2 \tag{7}
$$

$$
(G_{j_2, f_1})_d := (G_{j_2, f_1})_d + (g_{j_2, f_1})_d^2 \tag{8}
$$

最终我们的梯度更新为：

$$
(w_{j_1, f_2})_d := (w_{j_1, f_2})_d - \frac{\eta}{\sqrt{(G_{j_1, f_2})_d}} (g_{j_1, f_2})_d \tag{9}
$$

$$
(w_{j_2, f_1})_d := (w_{j_2, f_1})_d - \frac{\eta}{\sqrt{(G_{j_2, f_1})_d}} (g_{j_2, f_1})_d \tag{10}
$$

在参数初始化时 $\mathbb{w}$ 可以按 $[0, \frac{1}{\sqrt{k}}]$ 的均匀分布来初始化。 另外，在初始化时对特征的规一化处理也很重要。

最终的算法为：

#### Algorithm 1 Training FFM using SG

1. Let $G \in R^{n \times f \times k}$ be a tensor of all ones.
2. Run the following loop for t epoches
3. for $i \in \{1,\cdots, m \}$ do
    - Smaple a data point $(y, \mathbf{x})$
    - calculate $\kappa$
    - for $j_1 \in \{1, \cdots , n\} $ do
        - for $j_2 \in \{j_1 +1 \cdots , n\}$ do
            - calculate sub-gradient by (5) and (6)
            - for $d \in \{1, \cdots , k\} $ do
                - Update the gradient sum by (7) and (8)
                - Update by (9) and (10)

### 3.2 并行化训练

并行化训练请参考 参考[HOGWILD!: A Lock-Free Approach to Parallelizing Stochastic Gradient Descent](https://arxiv.org/abs/1106.5730) 并行设计的方案。

对于上面的算法，我们可从第 3 步中开始并行计算。

### 3.3 添加 Field 信息

通常在训练时都使用 LibSVM 提供的数据格式：

```
label feat1:val1 feat2:val2 ....
```

而对于 FFM，我们需要将特征所属的 Field 也加进来，加入 Field 后的格式为：

```
label field1:feat1:val1 field2:feat2:val2 ....
```

在加入 Field 的时候，我们需要考虑以下三种特征的处理方式：

1. Categorical Features

处理方法最简单

2. Numerical Features

3. Signle-field Features.

### 4. 实验

实验数据使用了 Criteo, Avazu 的数据。分别考虑了 $k, \lambda, \eta$ 的取值对模型的影响，以及并行计算时线程个数的选择。